<a href="https://colab.research.google.com/github/sviteribuben/ML_01/blob/main/homework_classification_2_les_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [16]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [17]:
wine_df = load_wine()
wine_df.data.shape

(178, 13)

In [18]:
# тримим фичи, оставляем только 9, 11, 12 фичи  

wine_dataset_1 = np.delete(wine_df.data, \
                           [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, ], 1)
wine_dataset_1.shape

(178, 3)

In [19]:
wine_df['target'].shape

(178,)

In [20]:
### YOUR CODE HERE ###
# делим дату
x_train, x_test, y_train, y_test = train_test_split(wine_dataset_1, 
                                                    wine_df['target'],
                                                    random_state=17)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [21]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [22]:
# включаем шафл, чтобы работал рандом стэйт, 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
skf.get_n_splits(x_train, y_train)
skf

StratifiedKFold(n_splits=5, random_state=17, shuffle=True)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [24]:
### YOUR CODE HERE ###
lr = LogisticRegression(C=1.0, multi_class='ovr', random_state=17,)

In [25]:
# Средняя доля правильных ответов на трэйне 

scores = cross_val_score(estimator=lr, X=x_train, y=y_train, 
                         cv=skf, scoring=None, n_jobs=-1).mean()

scores

0.8803418803418804

In [26]:
# Доля правильных ответов на тесте
lr_model = lr.fit(x_train, y_train)
lr_prediction = lr_model.predict(x_test)
accuracy_score(y_test, lr_prediction)

0.9777777777777777

### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [ ]:
from sklearn.svm import SVC

In [28]:
svc_mod = SVC(decision_function_shape='ovr', random_state=17)

In [30]:
# Доля правильных ответов на трэйне при использовании SVC
scores_svc = cross_val_score(estimator=svc_mod, X=x_train, y=y_train, 
                         cv=skf, scoring=None, n_jobs=-1).mean()

scores_svc

0.6923076923076923

In [38]:
# Доля правильных ответов на тесте при использовании SVC
svc_model = svc_mod.fit(x_train, y_train)
svc_prediction = svc_model.predict(x_test)
accuracy_score(y_test, svc_prediction)

0.6222222222222222

Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
### YOUR CODE HERE ###

# строим confusion matrix для модели логистической регрессии.
print('---МАТРИЦА ОШИБОК---')
print(confusion_matrix(y_test, lr_prediction))

---МАТРИЦА ОШИБОК---
[[ 9  0  0]
 [ 0 19  0]
 [ 0  1 16]]


модель путает третий класс один случай записала во второй.

In [36]:
# строим confusion matrix для метода опорных векторов
print('---МАТРИЦА ОШИБОК---')
print(confusion_matrix(y_test, svc_prediction))

---МАТРИЦА ОШИБОК---
[[ 9  0  0]
 [ 0 19  0]
 [ 2 15  0]]


модель путает также тертий класс и ниразу не классифицировала его правильно

### 5.
Для каждой модели выведите classification report.

In [37]:
### YOUR CODE HERE ###
# Добавляем zero_division=0 чтобы отключить предупреждение
target_names = ['class 0', 'class 1', 'class 2']
print('Logistic Regression')
print(classification_report(y_test, lr_prediction,\
                            zero_division=0, target_names=target_names,))
print('-'*60)
print('SVM')
print(classification_report(y_test, svc_prediction, \
                            zero_division=0, target_names=target_names,))

Logistic Regression
              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00         9
     class 1       0.95      1.00      0.97        19
     class 2       1.00      0.94      0.97        17

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

------------------------------------------------------------
SVM
              precision    recall  f1-score   support

     class 0       0.82      1.00      0.90         9
     class 1       0.56      1.00      0.72        19
     class 2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45

